In [1]:
'''
Checking Vector Jacobian Products For a simple neural net
'''

In [2]:
import torch
import torch.nn.functional as F
from torch.autograd.functional import vjp

In [3]:
# Later increase dimensions to include batches

#FORWARD PASS

x = torch.randn(3,1,dtype=torch.float)

In [4]:
W1 = torch.randn(4,3,dtype= torch.float)
b1 = torch.randn(4,1,dtype= torch.float)

In [5]:
print(W1.mm(x)+b1)

tensor([[ 0.1403],
        [ 0.6532],
        [-0.1662],
        [-0.4212]])


In [6]:
z = W1.mm(x)+b1

In [7]:
h = F.relu(z)

In [8]:
W2 = torch.randn(2,4,dtype= torch.float)
b2 = torch.randn(2,1,dtype= torch.float)
y = W2.mm(h) + b2
y.squeeze(-1)

tensor([ 2.2695, -0.7558])

In [9]:
t = torch.ones(2)

In [10]:
L = F.mse_loss(t,y)

/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  """Entry point for launching an IPython kernel.


In [11]:
print(L)

tensor(2.3472)


In [12]:
L_bar = 1

In [13]:
y_bar = vjp(func=F.mse_loss,inputs=(y,t))[1][0]
print(y_bar)

tensor([[ 1.2695],
        [-1.7558]])


/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/torch/autograd/functional.py:251: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  outputs = func(*inputs)


In [14]:
print(vjp(func=F.mse_loss,inputs=(y,t)))

(tensor(2.3472), (tensor([[ 1.2695],
        [-1.7558]]), tensor([0.2431, 0.2431])))


In [15]:
'''
Actually here derivative of t is wrong. It shud just be negative of y derivatives. It comes out right in case 
of batch products
'''

'\nActually here derivative of t is wrong. It shud just be negative of y derivatives. It comes out right in case \nof batch products\n'

In [16]:
def affine(W,h,b):
    return W.mm(h)+ b
print(vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[0])
print(vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[1])

tensor([[ 2.2695],
        [-0.7558]])
(tensor([[ 0.1781,  0.8292,  0.0000,  0.0000],
        [-0.2463, -1.1468, -0.0000, -0.0000]]), tensor([[-2.6816],
        [ 1.0106],
        [ 2.1407],
        [ 1.6628]]), tensor([[ 1.2695],
        [-1.7558]]))


In [17]:
print(h)

tensor([[0.1403],
        [0.6532],
        [0.0000],
        [0.0000]])


In [18]:
h_bar = vjp(func=affine,inputs=(W2,h,b2),v = y_bar)[1][1]
z_bar = vjp(func= F.relu, inputs= z,v = h_bar)
print(z_bar)

(tensor([[0.1403],
        [0.6532],
        [0.0000],
        [0.0000]]), tensor([[-2.6816],
        [ 1.0106],
        [ 0.0000],
        [ 0.0000]]))


In [19]:
'''
Dimensions and values of backward gradients correct. Only problem is ouput of func is calculated twice once in
forward pass and second in vjp. I will also have to take care of not updating variables untill both forward and 
backward passes are done.

Now trying a smaller network with batch size
'''

'\nDimensions and values of backward gradients correct. Only problem is ouput of func is calculated twice once in\nforward pass and second in vjp. I will also have to take care of not updating variables untill both forward and \nbackward passes are done.\n\nNow trying a smaller network with batch size\n'

In [20]:
batch_size = 10

x_ = torch.randn([batch_size,3,10],dtype=torch.float)
W_ = torch.randn([10,3],dtype= torch.float)
b_ = torch.randn([10,3,3], dtype = torch.float)

#torch.mm() works only for 2D matrices
#torch.matmul does batchwise multiplication if second tensor's dim > 2

In [21]:
z_ = x_.matmul(W_) + b_ 
print(z)

tensor([[ 0.1403],
        [ 0.6532],
        [-0.1662],
        [-0.4212]])


In [22]:
y_ = F.relu(z_)
print(y_)

tensor([[[ 5.7178,  1.0459,  0.0000],
         [ 3.9104,  0.0000,  0.0000],
         [ 0.3138,  0.0000,  1.6776]],

        [[ 0.0000,  2.8858,  3.5021],
         [ 0.9874,  3.1130,  1.7054],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  1.3910,  0.6841],
         [ 1.9745,  2.1534,  0.0000],
         [ 1.0571,  0.0000,  0.0000]],

        [[ 5.3951,  2.8337,  0.0000],
         [ 0.0000,  0.0000,  0.7631],
         [ 0.1244,  4.7911,  0.0000]],

        [[ 2.0741,  2.2804,  0.0000],
         [11.2952,  0.0000,  0.0000],
         [ 0.0000,  0.7330,  1.0997]],

        [[ 0.0000,  0.0000,  0.0000],
         [ 0.5447,  0.0000,  5.3743],
         [ 8.8495,  5.8250,  0.0000]],

        [[ 1.7818,  0.0000,  0.0000],
         [ 0.0000,  0.6814,  0.0000],
         [ 0.0000,  3.8262,  0.0000]],

        [[ 1.7551,  0.0000,  2.6309],
         [ 0.0000,  1.7675,  0.0000],
         [ 1.8795,  0.0000,  3.4326]],

        [[ 4.5406,  0.0365,  0.9930],
         [ 0.0000,  0.0000,  0.000

In [23]:
t_ = torch.ones([10,3,3])

In [24]:
L_ = F.mse_loss(y_,t_)

In [25]:
print(vjp(func = F.mse_loss,inputs = (y_,t_)))
#y_bar_ = vjp(func = F.mse_loss,inputs = (y_,t_))[1][0]
#print(y_bar_)
#print(y_bar_.size())

(tensor(4.4101), (tensor([[[ 1.0484e-01,  1.0205e-03, -2.2222e-02],
         [ 6.4675e-02, -2.2222e-02, -2.2222e-02],
         [-1.5249e-02, -2.2222e-02,  1.5059e-02]],

        [[-2.2222e-02,  4.1906e-02,  5.5603e-02],
         [-2.8070e-04,  4.6955e-02,  1.5677e-02],
         [-2.2222e-02, -2.2222e-02, -2.2222e-02]],

        [[-2.2222e-02,  8.6899e-03, -7.0202e-03],
         [ 2.1655e-02,  2.5630e-02, -2.2222e-02],
         [ 1.2694e-03, -2.2222e-02, -2.2222e-02]],

        [[ 9.7668e-02,  4.0748e-02, -2.2222e-02],
         [-2.2222e-02, -2.2222e-02, -5.2652e-03],
         [-1.9457e-02,  8.4247e-02, -2.2222e-02]],

        [[ 2.3868e-02,  2.8454e-02, -2.2222e-02],
         [ 2.2878e-01, -2.2222e-02, -2.2222e-02],
         [-2.2222e-02, -5.9339e-03,  2.2161e-03]],

        [[-2.2222e-02, -2.2222e-02, -2.2222e-02],
         [-1.0118e-02, -2.2222e-02,  9.7206e-02],
         [ 1.7443e-01,  1.0722e-01, -2.2222e-02]],

        [[ 1.7372e-02, -2.2222e-02, -2.2222e-02],
         [-2.2222e-0

In [26]:
# trying with 7 final output units

_x = torch.randn([10,3],dtype= torch.float)
_W = torch.randn([3,7],dtype= torch.float)
_b = torch.randn([10,7], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn([10,7], dtype= torch.float)

_y_bar = vjp(F.mse_loss,(_y,_t))
print(_y_bar)

#gradients are correct

(tensor(2.6546), (tensor([[ 6.2672e-03,  1.2650e-02, -1.9658e-02,  6.7980e-02,  6.4134e-02,
          8.3596e-03, -1.3857e-02],
        [ 9.9524e-03,  1.0229e-01, -4.2670e-03,  4.7433e-04, -1.3110e-02,
          1.1048e-01,  1.1469e-01],
        [ 1.7734e-01, -1.6613e-02,  6.5359e-02,  6.0111e-02, -6.5277e-02,
          1.4835e-02, -9.0519e-03],
        [-1.9503e-02,  8.7446e-03,  6.6337e-02, -7.2317e-03,  5.9847e-02,
         -1.6922e-02, -4.7051e-02],
        [-5.9387e-03,  3.4639e-02,  4.5419e-02, -4.2617e-03,  1.3630e-02,
         -2.3614e-02,  5.1468e-02],
        [ 4.3061e-03,  6.3390e-02,  2.2936e-02,  1.1105e-04, -2.5566e-02,
          5.8967e-02,  2.7806e-02],
        [-5.9983e-02, -6.0710e-03, -4.5048e-02,  2.1873e-02, -3.5698e-03,
         -8.6097e-03,  5.3809e-02],
        [-6.8750e-02,  3.9426e-02, -2.0179e-02, -4.0757e-02,  3.9555e-02,
         -8.7818e-03,  3.1005e-02],
        [ 8.2523e-04, -1.3954e-02,  3.3289e-02,  3.8400e-02,  5.1360e-02,
          3.6493e-03,  5.599

In [27]:
_x = torch.randn([2,3],dtype= torch.float)
_W = torch.randn([3,1],dtype= torch.float)
_b = torch.randn([2,1], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn(2, dtype= torch.float) #Fail
#_t = torch.randn([2,1], dtype= torch.float) #Pass . With exact shape it passes

_y_bar = vjp(F.mse_loss,(_y,_t))
print(_y_bar)


(tensor(2.8841), (tensor([[0.7892],
        [0.8819]]), tensor([-2.3133,  0.6422])))


In [28]:
'''
Alright I figured out the problem. when the shape of target variables is exactly same as to input then the gradients
come out right. but lets say if the shape of target is [2] and input is [2,1] then gradients arent correct because
there is another operation involved in the middle for reshaping.

I WILL HAVE TO BE VERY CAREFUL WITH SHAPES COZ broadcasting CHANGES GRADIENT when used for calculating loss in 
MSE as mentioned in the user warning..so, TARGET SHUD ALWAYS BE OF SAME DIMENSION 
ALSO, VJP ONLY TAKES TENSORS AS INPUT

Actually lets check the gradients using backward()
'''

#x = torch.rand(2)
#print(vjp(torch.reshape,(x,(-1*torch.ones(1)).data)))

_x = torch.randn([2,3],dtype= torch.float,requires_grad = True)
_W = torch.randn([3,1],dtype= torch.float, requires_grad = True)
_b = torch.randn([2,1], dtype = torch.float, requires_grad = True)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_y.retain_grad()
_t = torch.randn(2,requires_grad=True) # Pass only with exact shape [2,1]


_loss = F.mse_loss(_y,_t)
_loss.backward()

print(_t)
print(_y)

print(_t.grad)
print(_y.grad)


tensor([0.5871, 1.1182], requires_grad=True)
tensor([[0.],
        [0.]], grad_fn=<ReluBackward0>)
tensor([0.5871, 1.1182])
tensor([[-0.8526],
        [-0.8526]])


/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


In [29]:
_x = torch.randn([2,3],dtype= torch.float)
_W = torch.randn([3,1],dtype= torch.float)
_b = torch.randn([2,1], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn([2,1], dtype= torch.float)
_r = torch.randn([2,1], dtype= torch.float)

def loss(y,t,r):
    return torch.sum(torch.mul((y-t),(y-t)))/len(y)

_y_bar = vjp(func=loss,inputs=(_y,_t,_r))
print(_y_bar)

(tensor(0.3977), (tensor([[0.2783],
        [0.8473]]), tensor([[-0.2783],
        [-0.8473]]), tensor([[0.],
        [0.]])))


In [30]:

_x = torch.randn([2,3],dtype= torch.float)
_W = torch.randn([3,1],dtype= torch.float)
_b = torch.randn([2,1], dtype = torch.float)

_z = _x.matmul(_W) + _b
_y = F.relu(_z)
_t = torch.randn([2,1], dtype= torch.float) 

def dummy(y,t,*args):
    y = y*args[0]
    return F.mse_loss(y,t)

_y_bar = vjp(dummy,(_y,_t,_x,_W))
print(_y_bar)

(tensor(1.4710), (tensor([[0.4532],
        [0.0565]]), tensor([[1.6381],
        [0.1048]]), tensor([[-0.6377, -0.4361, -0.9107],
        [ 0.0015, -0.0216, -0.0043]]), tensor([[0.],
        [0.],
        [0.]])))


/home/akashe/anaconda3/envs/latest_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  # This is added back by InteractiveShellApp.init_path()
